### Rival posession overall

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
#df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')
df = pd.read_csv('/Users/luisalawson/Downloads/Sample Data/epl.csv', delimiter=';')

In [36]:
import pandas as pd

def group_plays(df):
    match_stats_list = []

    unique_matches = df['match_id'].unique()

    for currentmatch in unique_matches:
        match_df = df[df['match_id'] == currentmatch]

        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        home_plays = 0
        away_plays = 0
        home_passes = 0
        away_passes = 0

        current_team = None
        last_event_id = None
        in_play = False
        current_passes = 0

        for i, row in match_df.iloc[4:].iterrows():
            team_id = row['team_id']
            event_id = row['event_id']

            if not in_play:
                in_play = True
                current_team = team_id
                last_event_id = event_id
                current_passes = 0
            else:
                if team_id != current_team or event_id <= last_event_id:
                    # Play ends, record passes
                    if current_team == home_team_id:
                        home_plays += 1
                        home_passes += current_passes
                    else:
                        away_plays += 1
                        away_passes += current_passes
                    # New play starts
                    current_team = team_id
                    last_event_id = event_id
                    current_passes = 0
                else:
                    # Continue current play
                    last_event_id = event_id

            # Count successful passes in the current play
            if row['description'] == 'Pass' and row['outcome'] == 1:
                current_passes += 1
        
        # Account for the last play
        if in_play:
            if current_team == home_team_id:
                home_plays += 1
                home_passes += current_passes
            else:
                away_plays += 1
                away_passes += current_passes

        match_stats = {
            'match_id': currentmatch,
            'home_team':home_team_name,
            'away_team':away_team_name,
            'home_team_plays': home_plays,
            'away_team_plays': away_plays,
            'home_passes_per_play': home_passes / home_plays if home_plays > 0 else 0,
            'away_passes_per_play': away_passes / away_plays if away_plays > 0 else 0
        }

        match_stats_list.append(match_stats)
    
    return pd.DataFrame(match_stats_list)


In [39]:
df_grouped_plays = group_plays(df)

In [40]:
df_grouped_plays[(df_grouped_plays['home_team']=='Manchester United') | (df_grouped_plays['away_team']=='Manchester United')]

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play
1,442000,Everton,Manchester United,338,336,0.674556,1.800595
13,442012,Manchester United,Fulham,292,287,2.058219,1.355401
24,442023,Southampton,Manchester United,290,287,1.496552,1.912892
34,442033,Manchester United,Wigan Athletic,290,290,1.403448,1.224138
41,442040,Liverpool,Manchester United,319,317,1.338558,1.233438
54,442053,Manchester United,Tottenham Hotspur,318,316,2.069182,0.664557
63,442062,Newcastle United,Manchester United,326,328,1.254601,1.463415
72,442071,Manchester United,Stoke City,270,269,2.225926,1.048327
82,442081,Chelsea,Manchester United,313,309,1.239617,1.368932
92,442091,Manchester United,Arsenal,290,298,1.665517,1.848993


In [47]:
df_dc = pd.read_csv('/Users/luisalawson/Downloads/dc_results.csv')


In [48]:
df_dc['ranking_overall'] = df_dc['dc_overall'].rank(ascending=False)
df_dc['ranking_home'] = df_dc['dc_home'].rank(ascending=False)
df_dc['ranking_away'] = df_dc['dc_away'].rank(ascending=False)

In [52]:
def get_team_rankings(row, df_dc):
    home_team = row['home_team']
    away_team = row['away_team']

    home_team_ranking_overall = df_dc[df_dc['team_name'] == home_team]['ranking_overall'].values
    home_team_ranking_home = df_dc[df_dc['team_name'] == home_team]['ranking_home'].values
    home_team_ranking_away = df_dc[df_dc['team_name'] == home_team]['ranking_away'].values

    away_team_ranking_overall = df_dc[df_dc['team_name'] == away_team]['ranking_overall'].values
    away_team_ranking_home = df_dc[df_dc['team_name'] == away_team]['ranking_home'].values
    away_team_ranking_away = df_dc[df_dc['team_name'] == away_team]['ranking_away'].values

    return pd.Series({
        'home_team_ranking_overall': home_team_ranking_overall[0] if len(home_team_ranking_overall) > 0 else None,
        'home_team_ranking_home': home_team_ranking_home[0] if len(home_team_ranking_home) > 0 else None,
        'home_team_ranking_away': home_team_ranking_away[0] if len(home_team_ranking_away) > 0 else None,
        'away_team_ranking_overall': away_team_ranking_overall[0] if len(away_team_ranking_overall) > 0 else None,
        'away_team_ranking_home': away_team_ranking_home[0] if len(away_team_ranking_home) > 0 else None,
        'away_team_ranking_away': away_team_ranking_away[0] if len(away_team_ranking_away) > 0 else None
    })

In [53]:
df_grouped_plays[['home_team_ranking_overall', 'home_team_ranking_home', 'home_team_ranking_away', 
                   'away_team_ranking_overall', 'away_team_ranking_home', 'away_team_ranking_away']] = df_grouped_plays.apply(get_team_rankings, axis=1, df_dc=df_dc)

In [55]:
df_grouped_plays[(df_grouped_plays['home_team']=='Manchester United') | (df_grouped_plays['away_team']=='Manchester United')]

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_team_ranking_overall,home_team_ranking_home,home_team_ranking_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away
1,442000,Everton,Manchester United,338,336,0.674556,1.800595,13.0,13.0,14.0,5.0,6.0,1.0
13,442012,Manchester United,Fulham,292,287,2.058219,1.355401,5.0,6.0,1.0,6.0,5.0,4.0
24,442023,Southampton,Manchester United,290,287,1.496552,1.912892,3.0,4.0,2.0,5.0,6.0,1.0
34,442033,Manchester United,Wigan Athletic,290,290,1.403448,1.224138,5.0,6.0,1.0,7.0,9.0,6.0
41,442040,Liverpool,Manchester United,319,317,1.338558,1.233438,1.0,1.0,3.0,5.0,6.0,1.0
54,442053,Manchester United,Tottenham Hotspur,318,316,2.069182,0.664557,5.0,6.0,1.0,12.0,11.0,15.0
63,442062,Newcastle United,Manchester United,326,328,1.254601,1.463415,17.0,17.0,19.0,5.0,6.0,1.0
72,442071,Manchester United,Stoke City,270,269,2.225926,1.048327,5.0,6.0,1.0,8.0,8.0,8.0
82,442081,Chelsea,Manchester United,313,309,1.239617,1.368932,2.0,2.0,7.0,5.0,6.0,1.0
92,442091,Manchester United,Arsenal,290,298,1.665517,1.848993,5.0,6.0,1.0,4.0,3.0,5.0


In [71]:
import numpy as np

condition1 = (df_grouped_plays['home_passes_per_play'] > df_grouped_plays['away_passes_per_play'])
condition2 = (df_grouped_plays['home_team_ranking_overall'] < df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_ranking_home'] < df_grouped_plays['away_team_ranking_away'])

df_grouped_plays['correct'] = np.where(condition1 & condition2, 1, 0)

condition3 = (df_grouped_plays['home_passes_per_play'] < df_grouped_plays['away_passes_per_play'])
condition4 = (df_grouped_plays['home_team_ranking_overall'] > df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_ranking_home'] > df_grouped_plays['away_team_ranking_away'])

df_grouped_plays['correct'] = np.where(condition3 & condition4, 1, df_grouped_plays['correct'])

In [72]:
away_vis = df_grouped_plays['away_team_ranking_away']

In [75]:
# excel_file_path = '/Users/luisalawson/Downloads/pass_dc_comparison.xlsx'  

# df_grouped_plays.to_excel(excel_file_path, index=False)
# print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/pass_dc_comparison.xlsx


In [74]:
df_grouped_plays[(df_grouped_plays['home_team']=='Manchester United') | (df_grouped_plays['away_team']=='Manchester United')]

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_team_ranking_overall,home_team_ranking_home,home_team_ranking_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,correct
1,442000,Everton,Manchester United,338,336,0.674556,1.800595,13.0,13.0,14.0,5.0,6.0,1.0,1
13,442012,Manchester United,Fulham,292,287,2.058219,1.355401,5.0,6.0,1.0,6.0,5.0,4.0,1
24,442023,Southampton,Manchester United,290,287,1.496552,1.912892,3.0,4.0,2.0,5.0,6.0,1.0,1
34,442033,Manchester United,Wigan Athletic,290,290,1.403448,1.224138,5.0,6.0,1.0,7.0,9.0,6.0,1
41,442040,Liverpool,Manchester United,319,317,1.338558,1.233438,1.0,1.0,3.0,5.0,6.0,1.0,1
54,442053,Manchester United,Tottenham Hotspur,318,316,2.069182,0.664557,5.0,6.0,1.0,12.0,11.0,15.0,1
63,442062,Newcastle United,Manchester United,326,328,1.254601,1.463415,17.0,17.0,19.0,5.0,6.0,1.0,1
72,442071,Manchester United,Stoke City,270,269,2.225926,1.048327,5.0,6.0,1.0,8.0,8.0,8.0,1
82,442081,Chelsea,Manchester United,313,309,1.239617,1.368932,2.0,2.0,7.0,5.0,6.0,1.0,1
92,442091,Manchester United,Arsenal,290,298,1.665517,1.848993,5.0,6.0,1.0,4.0,3.0,5.0,1


### Rival posession in dangerous area